In [ ]:
import math
import torch
import gpytorch
import matplotlib.pyplot as plt

# Gaussian Process Regression

## Introduction: the GP prior model

### Setting up a GP model with [GPytorch](https://gpytorch.ai/)

The next cell demonstrates the most critical features of a user-defined Gaussian process model in GPyTorch.

- In contrast to many existing GP packages, GPyTorch does not provide full GP models for the user.
- Rather, it provides *the tools necessary to quickly construct one*.
- This is because we believe, analogous to building a neural network in standard PyTorch, it is important to have the flexibility to include whatever components are necessary: this allows the user great flexibility in designing custom models.

For most GP regression models, you will need to construct the following GPyTorch objects:

1. A **GP Model** (`gpytorch.models.ExactGP`).
1. A **Mean** - This defines the prior mean of the GP.
    - (If you don't know which mean to use, a `gpytorch.means.ConstantMean()` is a good place to start.
1. A **Kernel** - This defines the prior covariance of the GP.
    - If you don't know which kernel to use, a `gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())` is a good place to start.
  
### The GP Model
  
The components of a user built (Exact) GP model in GPyTorch are, broadly speaking:

1. An `__init__` method that takes the data and a likelihood, and constructs whatever objects are necessary for the model's `forward` method. This will most commonly include things like a mean module and a kernel module.

2. A `forward` method that takes in some $n \times d$ data `x` and returns a `MultivariateNormal` with the *prior* mean and covariance evaluated at `x`.
    - It returns a **MultivariateNormal** Distribution (`gpytorch.distributions.MultivariateNormal`), the object used to represent multivariate normal distributions. 
    - In other words, it returns the vector $\mu(x)$ and the $n \times n$ matrix $K_{xx}$ representing the prior mean and covariance matrix of the GP.
    
This specification leaves a large amount of flexibility when defining a model. For example, to compose two kernels via addition, you can either add the kernel modules directly:

```python
self.covar_module = ScaleKernel(RBFKernel() + LinearKernel())
```

Or you can add the outputs of the kernel in the forward method:

```python
covar_x = self.rbf_kernel_module(x) + self.white_noise_module(x)
```

### Model modes

Like most PyTorch modules, the `ExactGP` has a `.train()` and `.eval()` mode.
- `.eval()` mode is for computing predictions through the model posterior.
- `.train()` mode is for optimizing model hyperameters.

In [ ]:
# Exact GP model definition
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, x, y, mean_f, kernel_f, likelihood_f):
        super(ExactGPModel, self).__init__(x, y, likelihood_f)
        self.mean_module = mean_f
        self.covar_module = kernel_f
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

### Instantiations of the model

In [ ]:
# Dummy data, to later be replaced with real data, now used for plotting the prior GP model
x = torch.linspace(-1, 1, 100)
y = torch.ones(100)

In [ ]:
# Mean function
mean_f = gpytorch.means.ConstantMean()
# Kernel function
kernel_f = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

# Likelihood function
likelihood_f = gpytorch.likelihoods.GaussianLikelihood()

# GP model:
gp_model = ExactGPModel(
    x,
    y,
    mean_f,
    kernel_f,
    likelihood_f
)

In [ ]:
# Plotting the prior
with torch.no_grad():
    plt.figure()

    # The mean and covariance of the prior
    gp_model_prior = gp_model(x)

    # Plot the mean
    plt.plot(x.numpy(), gp_model_prior.mean.numpy(), 'k')
    # Plot the 95% confidence interval
    lower, upper = gp_model_prior.confidence_region()
    plt.fill_between(x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    plt.title('Prior GP model')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend(['Mean', 'Confidence interval'])
    plt.grid()

In [ ]:
# Plot samples from the prior
with torch.no_grad():
    plt.figure()
    for _ in range(5):
        sample = gp_model_prior.sample()
        # Plot the samples, changing the color each sample
        # to make it easier to see the samples
        plt.plot(x.numpy(), sample.numpy(),
                 color=plt.cm.viridis(torch.rand(1).item()), alpha=0.5)
    plt.title('Samples from the prior GP model')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid()
    plt.legend(['Samples $f(x)\sim GP$'])
plt.show()

In [ ]:
# Plot the prior GP model, along with some samples
# from the prior GP model
with torch.no_grad():
    plt.figure()
    for _ in range(5):
        sample = gp_model_prior.sample()
        # Plot the samples, changing the color each sample
        # to make it easier to see the samples
        plt.plot(x.numpy(), sample.numpy(),
                 color=plt.cm.viridis(torch.rand(1).item()), alpha=0.5)
    # Plot the mean
    plt.plot(x.numpy(), gp_model_prior.mean.numpy(), 'k')
    # Plot the 95% confidence interval
    lower, upper = gp_model_prior.confidence_region()
    plt.fill_between(x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    plt.title('Samples and the prior GP model')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid()
    plt.legend(['Samples $f(x)\sim GP$'])

# Replicate the above plot for different GP priors

Check out GPytorch documentation:

- [Different mean functions](https://docs.gpytorch.ai/en/stable/means.html)

- [Different kernel functions](https://docs.gpytorch.ai/en/stable/kernels.html)

In [ ]:
# Mean functions
# Constant mean function

# Linear mean function with bias

# Kernel functions
# RBF kernel

# Mattern kernel

# Periodic kernel

# Linear kernel

# Polynomial kernel

# Cosine kernel

# Likelihood function
likelihood_f = gpytorch.likelihoods.GaussianLikelihood()

In [ ]:
# Plot the prior GP model, along with some samples, from each prior GP model


## GP prior to posterior

In [ ]:
# Observe some data
x = torch.linspace(-1, 1, 50)
x_full = torch.linspace(-5, 5, 500)
# Observation noise
sigma = 0.1
y_linear = 2 * x + 1 + torch.randn(x.size()) * sigma

# Plot the data
plt.figure()
plt.scatter(x.numpy(), y_linear.numpy(), label='Linear data')
plt.title('Observed data')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Define a GP prior model

# Mean function
mean_f = gpytorch.means.ConstantMean()
# Kernel function
kernel_f = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

# Likelihood function
likelihood_f = gpytorch.likelihoods.GaussianLikelihood()

In [ ]:
# GP model with observed data:
gp_model_linear = ExactGPModel(
    x,
    y_linear,
    mean_f,
    kernel_f,
    likelihood_f
)

In [ ]:
# Plot the prior GP model
with torch.no_grad():
    plt.figure()

    # The mean and covariance of the prior, over full x
    gp_model_prior = gp_model_linear(x)

    # Plot the mean
    plt.plot(x.numpy(), gp_model_prior.mean.numpy(), 'k')
    # Plot the 95% confidence interval
    lower, upper = gp_model_prior.confidence_region()
    plt.fill_between(x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    plt.title('Prior GP model with linear data')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend(['Mean', 'Confidence interval'])
    plt.grid()


### Plot the posterior GP model

If we denote a test point (`test_x`) as `x*` with the true output being `y*`, then `model(test_x)` returns the model posterior distribution `p(f* | x*, X, y)`, for training data `X, y`.
    - This posterior is the distribution over the function we are trying to model, and thus quantifies our model uncertainty.

```python
f_preds = model(test_x)

f_mean = f_preds.mean
f_var = f_preds.variance
f_covar = f_preds.covariance_matrix
f_samples = f_preds.sample(sample_shape=torch.Size(1000,))
```

### Plot the posterior GP model along with some samples

In [ ]:
# GP model with observed data:
gp_model_linear = ExactGPModel(
    x,
    y_linear,
    mean_f,
    kernel_f,
    likelihood_f
)

# Set the model in evaluation mode
gp_model_linear.eval()

with torch.no_grad():
    plt.figure()

    # The mean and covariance of the posterior
    gp_model_posterior = gp_model_linear(x_full)

    # Plot the mean
    plt.plot(x_full.numpy(), gp_model_posterior.mean.numpy(), 'k')
    # Plot the 95% confidence interval
    lower, upper = gp_model_posterior.confidence_region()
    plt.fill_between(x_full.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    # Plot some samples from the posterior
    for _ in range(5):
        sample = gp_model_posterior.sample()
        # Plot the samples, changing the color each sample
        # to make it easier to see the samples
        plt.plot(x_full.numpy(), sample.numpy(),
                 color=plt.cm.viridis(torch.rand(1).item()), alpha=0.5)
    # Plot the observations
    plt.scatter(x.numpy(), y_linear.numpy(), color='red', marker='x')
    plt.title('Posterior GP model and samples with linear data')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend(['Mean', 'Confidence interval', 'Samples $f(x)\sim GP$'])
    plt.grid()

## Making predictions with the model

In the next cell, we make predictions with the model. To do this, we simply put the model and likelihood in eval mode, and call both modules on the test data.

If we denote a test point (`test_x`) as `x*` with the true output being `y*`, then `model(test_x)` returns the model posterior distribution `p(f* | x*, X, y)`, for training data `X, y`.
    - This posterior is the distribution over the function we are trying to model, and thus quantifies our model uncertainty.

In contrast, `likelihood(model(test_x))` gives us the posterior predictive distribution `p(y* | x*, X, y)` which is the probability distribution over the predicted output value.
    - By including the _likelihood noise_ which is the noise in your observation (e.g. due to noisy sensor), the prediction is over the observed value of the test point.

Thus, getting the predictive mean and variance, and then sampling functions from the GP at the given test points could be accomplished with calls like:


In [ ]:
# Plot the predictions of the GP model with linear data, along with some samples, incorporating the likelihood
gp_model_linear = ExactGPModel(
    x,
    y_linear,
    mean_f,
    kernel_f,
    likelihood_f
)
# Set the model in evaluation mode
gp_model_linear.eval()
# Set the likelihood in evaluation mode
likelihood_f.eval()

with torch.no_grad():
    plt.figure()

    # The mean and covariance of the predictions, over region of interest
    x_interest = torch.linspace(-2, 2, 400)
    gp_model_prediction = likelihood_f(gp_model_linear(x_interest))
    # Plot the mean
    plt.plot(x_interest.numpy(), gp_model_prediction.mean.numpy(), 'k')
    # Plot the 95% confidence interval
    lower, upper = gp_model_prediction.confidence_region()
    plt.fill_between(x_interest.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    # Plot some predicted samples
    for _ in range(5):
        sample = gp_model_prediction.sample()
        # Plot the samples, changing the color each sample
        # to make it easier to see the samples
        plt.plot(x_interest.numpy(), sample.numpy(),
                 color=plt.cm.viridis(torch.rand(1).item()), alpha=0.5)
    # Plot the observations
    plt.scatter(x.numpy(), y_linear.numpy(), color='red', marker='x')
    plt.title('GP model predictions and samples with linear data')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend(['Mean', 'Confidence interval', 'Samples $f(x)\sim GP$'])
    plt.grid()
plt.show()

## Training A GP model

In the next cell, we handle using Type-II MLE to train the hyperparameters of the Gaussian process.

In GPyTorch, we make use of the standard PyTorch optimizers as from `torch.optim`, and all trainable parameters of the model should be of type `torch.nn.Parameter`.
    - Because GP models directly extend `torch.nn.Module`, calls to methods like `model.parameters()` or `model.named_parameters()` function as you might expect coming from PyTorch.

In most cases, the boilerplate code below will work well. It has the same basic components as the standard PyTorch training loop:

1. Zero all parameter gradients
2. Call the model and compute the loss
3. Call backward on the loss to fill in gradients
4. Take a step on the optimizer

However, defining custom training loops allows for greater flexibility.
    - For example, it is easy to save the parameters at each step of training, or use different learning rates for different parameters .

### Training the GP model

In [ ]:
def train_gp_model(model, likelihood, train_x, train_y, training_iter=50):
    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1) 

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model) # Includes GaussianLikelihood parameters

    # Iterate in training
    for i in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, train_y)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
            i + 1, training_iter, loss.item(),
            model.covar_module.base_kernel.lengthscale.item(),
            model.likelihood.noise.item()
        ))
        optimizer.step()

    # Return the model and the likelihood
    return model, likelihood

In [ ]:
# Define a GP prior
gp_model_linear = ExactGPModel(
    x,
    y_linear,
    mean_f,
    kernel_f,
    likelihood_f
)

In [ ]:
# Plot the prior GP model
with torch.no_grad():
    plt.figure()

    # The mean and covariance of the prior
    gp_model_prior = gp_model_linear(x)

    # Plot the mean
    plt.plot(x.numpy(), gp_model_prior.mean.numpy(), 'k')
    # Plot the 95% confidence interval
    lower, upper = gp_model_prior.confidence_region()
    plt.fill_between(x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    plt.title('Prior GP model with linear data')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend(['Mean', 'Confidence interval'])
    plt.grid()

In [ ]:
# Plot the posterior GP model with linear data
gp_model_linear.eval()
likelihood_f.eval()
with torch.no_grad():
    plt.figure()

    # The mean and covariance of the posterior
    gp_model_posterior = gp_model_linear(x_full)

    # Plot the mean
    plt.plot(x_full.numpy(), gp_model_posterior.mean.numpy(), 'k')
    # Plot the 95% confidence interval
    lower, upper = gp_model_posterior.confidence_region()
    plt.fill_between(x_full.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    # Plot some samples from the posterior
    for _ in range(5):
        sample = gp_model_posterior.sample()
        # Plot the samples, changing the color each sample
        # to make it easier to see the samples
        plt.plot(x_full.numpy(), sample.numpy(),
                 color=plt.cm.viridis(torch.rand(1).item()), alpha=0.5)
    # Plot the observations
    plt.scatter(x.numpy(), y_linear.numpy(), color='red', marker='x')
    plt.title('Posterior GP model and samples with linear data')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend(['Mean', 'Confidence interval', 'Samples $f(x)\sim GP$'])
    plt.grid()
plt.show()  

In [ ]:
# Set the model and likelihood in training mode
gp_model_linear.train()
likelihood_f.train()

# Train the GP model
gp_model_linear_trained, likelihood_f_trained = train_gp_model(
    gp_model_linear,
    likelihood_f,
    x,
    y_linear,
    training_iter=50
)

In [ ]:
# Plot the prior, trained GP model
with torch.no_grad():
    plt.figure()

    # The mean and covariance of the prior
    gp_model_prior = gp_model_linear_trained(x)

    # Plot the mean
    plt.plot(x.numpy(), gp_model_prior.mean.numpy(), 'k')
    # Plot the 95% confidence interval
    lower, upper = gp_model_prior.confidence_region()
    plt.fill_between(x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    plt.title('Prior trained GP model with linear data')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend(['Mean', 'Confidence interval'])
    plt.grid()
plt.show()

In [ ]:
# Plot the posterior, trained GP model with linear data
gp_model_linear_trained.eval()
likelihood_f_trained.eval()

with torch.no_grad():
    plt.figure()

    # The mean and covariance of the posterior
    gp_model_posterior = gp_model_linear_trained(x_full)

    # Plot the mean
    plt.plot(x_full.numpy(), gp_model_posterior.mean.numpy(), 'k')
    # Plot the 95% confidence interval
    lower, upper = gp_model_posterior.confidence_region()
    plt.fill_between(x_full.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    # Plot some samples from the posterior
    for _ in range(5):
        sample = gp_model_posterior.sample()
        # Plot the samples, changing the color each sample
        # to make it easier to see the samples
        plt.plot(x_full.numpy(), sample.numpy(),
                 color=plt.cm.viridis(torch.rand(1).item()), alpha=0.5)
    # Plot the observations
    plt.scatter(x.numpy(), y_linear.numpy(), color='red', marker='x')
    plt.title('Posterior trained GP model and samples with linear data')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend(['Mean', 'Confidence interval', 'Samples $f(x)\sim GP$'])
    plt.grid()
plt.show()

In [ ]:
# Plot the predictions of the trained GP model with linear data: i.e., incorporate the likelihood
gp_model_linear_trained.eval()
likelihood_f_trained.eval()

with torch.no_grad():
    plt.figure()

    # The mean and covariance of the predictions
    gp_model_prediction = likelihood_f_trained(gp_model_linear_trained(x_full))
    # Plot the mean
    plt.plot(x_full.numpy(), gp_model_prediction.mean.numpy(), 'k')
    # Plot the 95% confidence interval
    lower, upper = gp_model_prediction.confidence_region()
    plt.fill_between(x_full.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    # Plot some predicted samples
    for _ in range(5):
        sample = gp_model_prediction.sample()
        # Plot the samples, changing the color each sample
        # to make it easier to see the samples
        plt.plot(x_full.numpy(), sample.numpy(),
                 color=plt.cm.viridis(torch.rand(1).item()), alpha=0.5)
    # Plot the observations
    plt.scatter(x.numpy(), y_linear.numpy(), color='red', marker='x')
    plt.title('Trained GP model predictions and samples with linear data')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend(['Mean', 'Confidence interval', 'Samples $f(x)\sim GP$'])
    plt.grid()
plt.show()

### Replicate the above with different data and GP models

- How does the prior mean and covariance function influence the prior and the posterior?

- How does the posterior change close and far from data?

- How does the prior change before and after training?

- How does the posterior change before and after training?

- What happens with different datasets?

- How does the likelihood function affect predictions?